In [14]:
import os
from typing import Annotated, TypedDict
from dotenv import load_dotenv

# LangChain components
from langchain_community.chat_models import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage
from pydantic import BaseModel, Field

# LangGraph components
from langgraph.graph import StateGraph
from langgraph.graph.message import AnyMessage, add_messages
from langgraph.prebuilt import ToolNode

# Pretty printing for console output
from rich.console import Console

# ------------------------------------------------------------
# Load environment variables (API keys, configurations, etc.)
# ------------------------------------------------------------
load_dotenv()
console = Console()
print("Environment variables loaded and tracing is set up.")

from langchain_core.messages import HumanMessage

# ------------------------------------------------------------
# Agent State Definition
# ------------------------------------------------------------
class AgentState(TypedDict):
    """
    The state passed through the LangGraph.
    - messages: a running list of messages exchanged between user, agent, and tools.
    - Annotated[...] with add_messages tells LangGraph to append new messages automatically.
    """
    messages: Annotated[list[AnyMessage], add_messages]

print("AgentState TypedDict defined.")

# ------------------------------------------------------------
# Agent Node - Core LLM Node
# ------------------------------------------------------------
def agent_node(state: AgentState):
    """
    Main reasoning node.
    - Receives all past messages in `state["messages"]`
    - Calls the LLM to decide the next action
    - Returns the new message so LangGraph can continue routing
    - No bind_tools required; ToolNode handles tool execution automatically
    """
    console.print("--- AGENT: Thinking... ---")
    resp = llm.invoke(state["messages"])
    return {"messages": [resp]}

# ------------------------------------------------------------
# Configure the LLM (Ollama local model)
# ------------------------------------------------------------
# Ensure you have pulled the model locally:
#   ollama pull llama3
llm = ChatOllama(
    model="llama3",   # can be replaced with "llama3.1", "mistral", etc.
    temperature=0,    # deterministic behavior
)

# ------------------------------------------------------------
# Tool node setup
# ------------------------------------------------------------
# NOTE: You must define `tools` somewhere above this line.
# ToolNode automatically handles calling whichever tool the AI requests.
tool_node = ToolNode(tools)

# ------------------------------------------------------------
# Router Function - Controls graph branching
# ------------------------------------------------------------
def router_function(state: AgentState) -> str:
    """
    Routing logic for the graph.
    - If the last LLM message contains a tool call → route to tool execution
    - Otherwise → finish the graph run
    """
    last = state["messages"][-1]

    # Detecting tool calls in the LLM output
    if getattr(last, "tool_calls", None):
        console.print("--- ROUTER: tool call requested ---")
        return "call_tool"

    console.print("--- ROUTER: finished ---")
    return "__end__"

# ------------------------------------------------------------
# Build the LangGraph
# ------------------------------------------------------------
graph = StateGraph(AgentState)

# Add nodes:
graph.add_node("agent", agent_node)   # LLM decision node
graph.add_node("call_tool", tool_node)  # Executes tools

# Set entry point (first node to run)
graph.set_entry_point("agent")

# Conditional routing:
#   agent → call_tool (if tool call)
#   agent → __end__   (if no tool call)
graph.add_conditional_edges("agent", router_function)

# After a tool is executed, return to agent for next reasoning step
graph.add_edge("call_tool", "agent")

# Compile graph into a runnable app
tool_agent_app = graph.compile()

# ------------------------------------------------------------
# Run the agent with a sample query
# ------------------------------------------------------------
user_query = "What were the main announcements from Amazon's latest Invent event?"

# Initial state for LangGraph
initial_input = {"messages": [HumanMessage(content=user_query)]}

console.print(f"[bold cyan]🚀 Starting agent for query:[/bold cyan] {user_query}")

# Stream results as they come in (token/step streaming)
for chunk in tool_agent_app.stream(initial_input, stream_mode="values"):
    # Pretty print each incremental output message
    chunk["messages"][-1].pretty_print()
    console.print("\n---\n")

console.print("[bold green]✅ Done![/bold green]")


Environment variables loaded and tracing is set up.
AgentState TypedDict defined.


🚀 Starting agent for query: What were the main announcements from Amazon's latest Invent event?

================================ Human Message =================================

What were the main announcements from Amazon's latest Invent event?


---

--- AGENT: Thinking... ---

--- ROUTER: finished ---

================================== Ai Message ==================================

Amazon's re:Mars 2022, also known as the "Invent" event, took place on September 28-30, 2022. The event showcased the company's latest innovations and advancements in areas like robotics, artificial intelligence (AI), computer vision, and more. Here are some of the main announcements from the event:

1. **SageMaker Autopilot**: Amazon SageMaker Autopilot is a new service that automates the process of building, training, and deploying machine learning models. It uses reinforcement learning to optimize model performance and reduce costs.
2. **Amazon Robotics L6**: The company unveiled its latest robot, the L6, designed for warehouse automation. This robot can navigate complex environments, pick items from shelves, and perform tasks like box packing and labeling.
3. **Sumerian 3D Authoring Tool**: Amazon Sumerian is a cloud-based platform that enables users to create interactive, 3D experiences without requi

---

✅ Done!

In [20]:
import json
import re

def build_conversation_trace(messages):
    trace_lines = []
    for m in messages:
        role = getattr(m, "type", "unknown")
        content = getattr(m, "content", "")
        tool_calls = getattr(m, "tool_calls", "")
        trace_lines.append(f"{role.upper()}:\n{content}\n{tool_calls}\n")
    return "\n".join(trace_lines)


def build_eval_prompt(conversation_trace: str):
    return f"""
You are a rigorous evaluator of AI agent behaviors and tool use.

Evaluate the agent's behavior based ONLY on the conversation trace below.

Return a JSON object with the following keys:

- tool_selection_score: integer from 1–5  
- tool_input_score: integer from 1–5  
- synthesis_quality_score: integer from 1–5  
- overall_score: integer from 1–5  
- justification: a concise explanation (string)

CRITICAL RULES:
- Output valid JSON ONLY
- Do not add comments, markdown, code fences, or explanations
- All values must match the required types

Conversation Trace:
--------------------
{conversation_trace}
--------------------

Return ONLY the JSON.
"""

 

def evaluate_run(final_state, llm, console=None):
    # Build trace
    conversation_trace = build_conversation_trace(final_state["messages"])
    
    # Build prompt
    eval_prompt = build_eval_prompt(conversation_trace)

    # Call evaluator LLM
    response = llm.invoke(eval_prompt)
    raw = response.content

    try:
        parsed = json.loads(raw)
    except:
        parsed = repair_json(raw)

    evaluation = ToolUseEvaluation(**parsed)

    if console:
        console.print("\n[bold magenta]--- Evaluation ---[/bold magenta]")
        console.print(evaluation.model_dump())

    return evaluation
final_state = tool_agent_app.invoke(initial_input)
(final_state['messages'][-1].pretty_print())
evaluation = evaluate_run(final_state, llm, console)


--- AGENT: Thinking... ---

--- ROUTER: finished ---

--- Evaluation ---

{
    'tool_selection_score': 5,
    'tool_input_score': 4,
    'synthesis_quality_score': 5,
    'justification': "The AI agent effectively selected relevant tools and provided accurate information about 
Amazon's Invent event, with minor issues in tool input. The synthesis quality is excellent, providing a clear 
summary of the main announcements."
}